# PyBullet을 이용한 진자(Pendulum) 강화학습 환경

MATLAB의 진자 예제와 유사한 강화학습 환경을 PyBullet으로 구성

- **상태**: `[cos(theta), sin(theta), theta_dot]`
- **행동**: 진자 관절에 가하는 토크 (-2.0 ~ 2.0)
- **보상**: 진자가 위로 설수록, 각속도와 토크가 작을수록 높은 보상 획득.

### 1단계: URDF 파일 생성 (Code)

Jupyter Notebook의 매직 명령어 %%writefile을 사용하면 현재 디렉토리에 바로 파일을 생성할 수 있어 편리하다고 함!

In [1]:
from pybullet_examples.createObstacleCourse import width
%%writefile models/pendulum.urdf
<?xml version="1.0"?>
<robot name="pendulum">

  <link name="base">
    <inertial>
      <mass value="0.1"/>
      <inertia ixx="0.0001" ixy="0.0" ixz="0.0" iyy="0.0001" iyz="0.0" izz="0.0001"/>
    </inertial>
    <visual>
      <geometry>
        <cylinder length="0.1" radius="0.01"/>
      </geometry>
      <material name="blue">
        <color rgba="0 0 1 1"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <cylinder length="0.1" radius="0.01"/>
      </geometry>
    </collision>
  </link>

  <link name="pole">
    <inertial>
      <mass value="1.0"/>
      <inertia ixx="0.083" ixy="0.0" ixz="0.0" iyy="0.083" iyz="0.0" izz="0.005"/>
    </inertial>
    <visual>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
      <material name="red">
          <color rgba="1 0 0 1"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <cylinder length="1" radius="0.05"/>
      </geometry>
    </collision>
  </link>

  <joint name="hinge" type="revolute">
    <parent link="base"/>
    <child link="pole"/>
    <axis xyz="0 1 0"/>
    <limit effort="3" lower="-3.1415" upper="3.1415" velocity="15"/>
    <origin rpy="0 0 0" xyz="0 0 0.5"/>
  </joint>

</robot>

Overwriting models/pendulum.urdf


### 2단계: 라이브러리 임포트 (Code)

환경 구성과 테스트에 필요한 라이브러리들을 불러오기

In [1]:
import gymnasium as gym
from gymnasium import spaces
import pybullet as p
import pybullet_data
import numpy as np
import time
import cv2
print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


pybullet build time: May 17 2025 21:05:07


### 3단계: PyBullet 진자 환경 클래스 정의

핵심이 되는 gym.Env 상속 클래스 정의

In [40]:
class PyBulletPendulumEnv(gym.Env):
    metadata = {"render_modes": ["human"]}

    def __init__(self, render_mode='human', video_path=None):
        super(PyBulletPendulumEnv, self).__init__()

        self.render_mode = render_mode
        self.video_path = video_path
        self.frames=[]
        
        if self.render_mode == 'human':
            # 이전에 연결된 세션이 있다면 종료 후 새로 연결
            try:
                p.disconnect()
            except p.error:
                pass
            self.client = p.connect(p.GUI)
        else:
            self.client = p.connect(p.DIRECT)

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.8)

        # URDF 파일 로드 (같은 디렉토리에 생성됨)
        self.urdf_path = "models/pendulum.urdf"
        print(f"Loading URDF from: {self.urdf_path}")
        self.pendulumId = p.loadURDF(self.urdf_path)

        # 행동 공간 (Torque)
        self.max_torque = 2.0
        self.action_space = spaces.Box(
            low=-self.max_torque,
            high=self.max_torque,
            shape=(1,),
            dtype=np.float32
        )

        # 관측 공간 [cos(theta), sin(theta), theta_dot]
        high = np.array([1.0, 1.0, 15.0], dtype=np.float32) # max velocity from URDF
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        
        self.step_counter = 0
        self.width=640
        self.height=480

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.step_counter = 0
        self.frames = []

        init_angle = self.np_random.uniform(low=-np.pi, high=np.pi)
        init_velocity = self.np_random.uniform(low=-1.0, high=1.0)
        # p.resetJointState(self.pendulumId, 0, targetValue=init_angle, targetVelocity=init_velocity)
        p.resetJointState(self.pendulumId, 0, targetValue=init_angle, targetVelocity=0)

        p.setJointMotorControl2(
            bodyUniqueId=self.pendulumId,
            jointIndex=0,
            controlMode=p.TORQUE_CONTROL,
            force=0
        )

        return self._get_obs(), {}

    def step(self, action):
        torque = action[0] * self.max_torque
        print(f"action torque={torque}")

        p.setJointMotorControl2(
            bodyUniqueId=self.pendulumId,
            jointIndex=0,
            controlMode=p.TORQUE_CONTROL,
            force=torque
        )
        p.stepSimulation()
        
        if self.render_mode != 'human' and self.video_path is not None:
            self._capture_frame()
            
        obs = self._get_obs()
        reward = self._calculate_reward(obs, action)
        
        self.step_counter += 1
        truncated = self.step_counter >= 200
        terminated = False

        return obs, reward, terminated, truncated, {}
    
    def _capture_frame(self):
        width, height = self.width, self.height
        img = p.getCameraImage(self.width, self.height, renderer=p.ER_BULLET_HARDWARE_OPENGL)
        rgb_pixels = np.array(img[2]).reshape((height, width, 4))[:,:,:3]
        self.frames.append(rgb_pixels)
    
    def render(self):
        if self.render_mode == 'human':
            pass
        elif self.render_mode == 'rgb_array':
            if len(self.frames) > 0:
                return self.frames[-1]
            else:
                self._capture_frame()
                return self.frames[-1]
            
    def _get_obs(self):
        joint_state = p.getJointState(self.pendulumId, 0)
        theta, theta_dot = joint_state[0], joint_state[1]
        theta = (theta + np.pi) % (2 * np.pi) - np.pi # 각도 정규화
        return np.array([np.cos(theta), np.sin(theta), theta_dot], dtype=np.float32)

    def _calculate_reward(self, obs, action):
        cos_theta, sin_theta, theta_dot = obs
        theta = np.arctan2(sin_theta, cos_theta)
        torque = action[0]
        reward = -(theta**2 + 0.1 * theta_dot**2 + 0.001 * torque**2)
        return reward

    def close(self):
        if self.video_path is not None and len(self.frames) > 0:
            self._save_video()
        p.disconnect(self.client)
    
    def _save_video(self):
        print(f"Saving video to: {self.video_path} ...")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer=cv2.VideoWriter(self.video_path, fourcc, 20.0, (self.width, self.height))
        for frame in self.frames:
            bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            video_writer.write(bgr_frame)
        video_writer.release()
        print(f"Video saved")

print("PyBulletPendulumEnv 클래스 정의 완료!")

PyBulletPendulumEnv 클래스 정의 완료!


### 4단계: 환경 테스트

정의된 환경을 실제로 생성하고, 무작위 행동을 주면서 어떻게 동작하는지 테스트
우선, 불러온 펜듈럼이 시뮬에서 제대로 돌아가는지나 보자. 

In [41]:
# 환경 생성
env = PyBulletPendulumEnv(render_mode='non-human')
print("환경 테스트 시작...")

try:
    obs, info = env.reset()
    print("sim start")
    
    for _ in range (240*5):
        action=[2.0]
        envReturn = env.step(action)

        print(f"observation={envReturn[0]}")
        
        time.sleep(1./240.)
finally:
    print("sim end")
    env.close()

print("\n환경 테스트 종료.")

Loading URDF from: models/pendulum.urdf
b3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
Unknown geometry type:b3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
circleb3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
Could not parse visual element for Link:b3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
baseb3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
failed to parse link

error: Cannot load URDF file.

구성한 시뮬이 잘 작동하는 것을 확인한다면, policy를 세우고 학습을 진행하자. 

In [4]:
# 환경 생성
env = PyBulletPendulumEnv(render_mode='rgb_array',video_path="video/test.mp4")
obs, info = env.reset()

print("환경 테스트 시작...")

for episode in range(1): # 3번의 에피소드 테스트
    obs, info = env.reset()
    total_reward = 0
    
    for i in range(240*5):
        # 무작위 행동 샘플링
        # action = env.action_space.sample()
        action = [1.0]
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward

        # # GUI 환경을 위한 딜레이
        # time.sleep(1./240.)
        
        # if terminated or truncated: # 이거 때문에 for에서 길이 늘려도 빨리 끝나는듯. 
        #     break
            
    print(f"에피소드 {episode + 1}: 총 보상 = {total_reward:.2f}")

print("\n환경 테스트 종료.")

# 환경 종료
env.close()
print("PyBullet 연결이 종료되었습니다.")

Loading URDF from: models/pendulum.urdf
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
base환경 테스트 시작...
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque=2.0
action torque

In [39]:
import pybullet as p
import pybullet_data
import time
import cv2
import numpy as np

width, height = 640, 480
frames = []

p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# planeId = p.loadURDF("plane.urdf")  # 기본 바닥 평면 로드
urdf_path = "models/pendulum.urdf"
print(urdf_path)
p.setGravity(0, 0, -9.8)

pendulum_id = p.loadURDF(urdf_path, basePosition=[0, 0, 1], useFixedBase=True)
initial_angle = 3.14 / 7
p.resetJointState(
    bodyUniqueId=pendulum_id,
    jointIndex=0, # hinge 조인트
    targetValue=initial_angle,
    targetVelocity=0.0    
)

for step in range(240*5):
    p.setJointMotorControl2(bodyUniqueId=pendulum_id,
                            jointIndex=0,
                            controlMode=p.TORQUE_CONTROL,
                            force=0)
    p.stepSimulation()
    
    img = p.getCameraImage(width, height, renderer=p.ER_TINY_RENDERER)
    rgb = np.array(img[2]).reshape((height, width, 4))[:,:,:3]
    frames.append(rgb)
    
    if step % 100 == 0:
        joint_state = p.getJointState(pendulum_id, 0)
        position, velocity, _, _ = joint_state
        print(f"step {step}: joint Position = {position:.3f}, velocity = {velocity:.3f}")

video = cv2.VideoWriter('video/test.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (width, height))
for frame in frames:
    video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
video.release()

p.disconnect()

models/pendulum.urdf
b3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
Unknown geometry type:b3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
circleb3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
Could not parse visual element for Link:b3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
baseb3Error[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,121]:
failed to parse link

error: Cannot load URDF file.